In [6]:
class StreamCipher:

    
    def __init__(self, alphabet=None):


        if alphabet is None:
            self.alphabet = " абвгдежзийклмнопрстуфхцчшщъыьэюя"
        else:
            self.alphabet = alphabet
        
        self.modulus = len(self.alphabet)
    
    def text_to_numbers(self, text):


        numbers = []
        for char in text.lower():
            if char in self.alphabet:
                numbers.append(self.alphabet.index(char))
            else:
                numbers.append(0)
        return numbers
    
    def numbers_to_text(self, numbers):

        text = []
        for num in numbers:
            if 0 <= num < len(self.alphabet):
                text.append(self.alphabet[num])
            else:
                text.append(' ')  
        return ''.join(text)
    
    def encrypt(self, plaintext, gamma):

        plain_numbers = self.text_to_numbers(plaintext)
        gamma_numbers = self.text_to_numbers(gamma)

        if len(gamma_numbers) < len(plain_numbers):
          
            repeated_gamma = []
            while len(repeated_gamma) < len(plain_numbers):
                repeated_gamma.extend(gamma_numbers)
            gamma_numbers = repeated_gamma[:len(plain_numbers)]
        else:
            gamma_numbers = gamma_numbers[:len(plain_numbers)]
     
        cipher_numbers = []
        for p, g in zip(plain_numbers, gamma_numbers):
            cipher_num = (p + g) % self.modulus
            cipher_numbers.append(cipher_num)
        
        return self.numbers_to_text(cipher_numbers)
    
    def decrypt(self, ciphertext, gamma):

        cipher_numbers = self.text_to_numbers(ciphertext)
        gamma_numbers = self.text_to_numbers(gamma)
        

        if len(gamma_numbers) < len(cipher_numbers):

            repeated_gamma = []
            while len(repeated_gamma) < len(cipher_numbers):
                repeated_gamma.extend(gamma_numbers)
            gamma_numbers = repeated_gamma[:len(cipher_numbers)]
        else:
            gamma_numbers = gamma_numbers[:len(cipher_numbers)]

        plain_numbers = []
        for c, g in zip(cipher_numbers, gamma_numbers):
            plain_num = (c - g) % self.modulus
            plain_numbers.append(plain_num)

        return self.numbers_to_text(plain_numbers)




In [7]:
class LCGGenerator:

    def __init__(self, a, seed, b, m):

        self.a = a
        self.current = seed
        self.b = b
        self.m = m
    
    def next(self):

        self.current = (self.a * self.current + self.b) % self.m
        return self.current
    
    def generate_sequence(self, length):

        sequence = [self.current]
        for _ in range(length - 1):
            sequence.append(self.next())
        return sequence




In [8]:
def test_stream_cipher():

    

    russian_alphabet = " абвгдежзийклмнопрстуфхцчшщъыьэюя"
    cipher = StreamCipher(russian_alphabet)
    

    plaintext = "приказ"  
    gamma = "гамма"       
    

    print(f"Открытый текст: {plaintext}")
    print(f"Потоковой шифр: {gamma}")
    print(f"Модуль: {cipher.modulus}")
    print()
    

    plain_numbers = cipher.text_to_numbers(plaintext)
    gamma_numbers = cipher.text_to_numbers(gamma)
    
    print("Последовательность чисел в открытом тексте:", plain_numbers)
    print("Последовательность чисел в потоковом шифре:", gamma_numbers)
    print()
    

    encrypted = cipher.encrypt(plaintext, gamma)
    encrypted_numbers = cipher.text_to_numbers(encrypted)
    

    for i, (p_num, g_num) in enumerate(zip(plain_numbers, gamma_numbers)):
        c_num = (p_num + g_num) % cipher.modulus
        print(f"  {p_num} + {g_num} mod {cipher.modulus} = {c_num} -> '{russian_alphabet[c_num]}'")
    
    print(f"Зашифрованная последовательность чисел: {encrypted_numbers}")
    print()
    

    decrypted = cipher.decrypt(encrypted, gamma)
    decrypted_numbers = cipher.text_to_numbers(decrypted)
    

    for i, (c_num, g_num) in enumerate(zip(encrypted_numbers, gamma_numbers)):
        p_num = (c_num - g_num) % cipher.modulus
        print(f"  {c_num} - {g_num} mod {cipher.modulus} = {p_num} -> '{russian_alphabet[p_num]}'")
    
    print(f"Расшифровка последовательности чисел: {decrypted_numbers}")
    print()




In [9]:
def test_lcg_generator():

    
    a = 7       
    seed = 1 
    b = 3       
    m = 33      
    lcg = LCGGenerator(a, seed, b, m)
    

    sequence = lcg.generate_sequence(10)
    print(f"Параметры: a={a}, seed={seed}, b={b}, m={m}")
    print(f"Сгенерированная последовательность: {sequence}")
    

    cipher = StreamCipher()
    

    plaintext = "тестовый текст"
    lcg_sequence = lcg.generate_sequence(len(plaintext))
    

    gamma_text = cipher.numbers_to_text(lcg_sequence)
    
    print(f"\nОткрытый текст: {plaintext}")
    print(f"Потоковой шифр LCG: {gamma_text}")
    print(f"Последовательность чисел LCG: {lcg_sequence}")
    

    encrypted = cipher.encrypt(plaintext, gamma_text)
    print(f"Результат шифрования: {encrypted}")
    

    lcg2 = LCGGenerator(a, seed, b, m)
    lcg_sequence2 = lcg2.generate_sequence(len(encrypted))
    gamma_text2 = cipher.numbers_to_text(lcg_sequence2)
    
    decrypted = cipher.decrypt(encrypted, gamma_text2)
    print(f"Результат расшифровки: {decrypted}")




In [10]:
if __name__ == "__main__":
    test_stream_cipher()
    print("\n" + "="*50 + "\n")
    test_lcg_generator()

Открытый текст: приказ
Потоковой шифр: гамма
Модуль: 33

Последовательность чисел в открытом тексте: [16, 17, 9, 11, 1, 8]
Последовательность чисел в потоковом шифре: [4, 1, 13, 13, 1]

  16 + 4 mod 33 = 20 -> 'у'
  17 + 1 mod 33 = 18 -> 'с'
  9 + 13 mod 33 = 22 -> 'х'
  11 + 13 mod 33 = 24 -> 'ч'
  1 + 1 mod 33 = 2 -> 'б'
Зашифрованная последовательность чисел: [20, 18, 22, 24, 2, 12]

  20 - 4 mod 33 = 16 -> 'п'
  18 - 1 mod 33 = 17 -> 'р'
  22 - 13 mod 33 = 9 -> 'и'
  24 - 13 mod 33 = 11 -> 'к'
  2 - 1 mod 33 = 1 -> 'а'
Расшифровка последовательности чисел: [16, 17, 9, 11, 1, 8]



Параметры: a=7, seed=1, b=3, m=33
Сгенерированная последовательность: [1, 10, 7, 19, 4, 31, 22, 25, 13, 28]

Открытый текст: тестовый текст
Потоковой шифр LCG: ыайжтгюхшмыайж
Последовательность чисел LCG: [28, 1, 10, 7, 19, 4, 31, 22, 25, 13, 28, 1, 10, 7]
Результат шифрования: нжыщажщяшяалыщ
Результат расшифровки: мэфжэигжлг бфж
